<a href="https://colab.research.google.com/github/albertoxamin/telegram-deep-fakes-bot/blob/master/deep_fake_telegram.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Demo for paper "First Order Motion Model for Image Animation"

**Clone repository**

In [1]:
!git clone https://github.com/AliaksandrSiarohin/first-order-model

fatal: destination path 'first-order-model' already exists and is not an empty directory.


In [2]:
cd first-order-model

/content/first-order-model


**Mount your Google drive folder on Colab**

In [3]:
from google.colab import drive
drive.mount('/content/gdrive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/gdrive


**Add folder https://drive.google.com/drive/folders/1kZ1gCnpfU0BnpdU47pLM_TQ6RypDDqgw?usp=sharing  to your google drive.**

**Load driving video and source image**

In [0]:
import imageio
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.animation as animation
from skimage.transform import resize
from IPython.display import HTML
import warnings
warnings.filterwarnings("ignore")

**Create a model and load checkpoints**

In [0]:
from demo import load_checkpoints
from demo import make_animation
from skimage import img_as_ubyte
generator, kp_detector = load_checkpoints(config_path='config/vox-256.yaml', 
                            checkpoint_path='/content/gdrive/My Drive/first-order-motion-model/vox-cpk.pth.tar')

In [6]:
!pip install pyTelegramBotAPI

     |████████████████████████████████| 71kB 6.2MB/s 
  Created wheel for pyTelegramBotAPI: filename=pyTelegramBotAPI-3.6.7-cp36-none-any.whl size=47178 sha256=43a66743a5727be576aea609521f25218e5b6e354da3b9513e7563606224a930
  Stored in directory: /root/.cache/pip/wheels/23/40/18/8a34153f95ef0dc19e3954898e5a5079244b76a8afdd7d0ec5
Successfully built pyTelegramBotAPI


In [7]:
import telebot
import requests
import shutil

API_TOKEN=""

bot = telebot.TeleBot(API_TOKEN)

@bot.message_handler(commands=['start', 'help'])
def send_welcome(message):
	bot.reply_to(message, "Hi send me a photo (with square aspect ratio), you will then be able to use this face")


@bot.message_handler(content_types=['photo'])
def download_pic(message):
  file_info = bot.get_file(message.photo[0].file_id)
  file = requests.get('https://api.telegram.org/file/bot{0}/{1}'.format(API_TOKEN, file_info.file_path))
  id = message.chat.id;
  open(f'../src{id}.jpg', 'wb').write(file.content)
  bot.reply_to(message, "Perfect, now send me a video note! (for best results keep head movements to a minimum and keep a static background)")

@bot.message_handler(content_types=['video_note'])
def download_video(message):
  id = message.chat.id;
  file_info = bot.get_file(message.video_note.file_id)
  print('https://api.telegram.org/file/bot{0}/{1}'.format(API_TOKEN, file_info.file_path));
  file = requests.get('https://api.telegram.org/file/bot{0}/{1}'.format(API_TOKEN, file_info.file_path))
  open(f'../target{id}.mp4', 'wb').write(file.content)
  bot.reply_to(message, "I'm generating the deep fake...")
  bot.send_chat_action(id, 'record_video')
  source_image = imageio.imread(f'../src{id}.jpg')
  source_image = resize(source_image, (256, 256))[..., :3]
  driving_video = imageio.mimread(f'../target{id}.mp4')
  driving_video = [resize(frame, (256, 256))[..., :3] for frame in driving_video]
  predictions = make_animation(source_image, driving_video, generator, kp_detector, relative=True)
  imageio.mimsave(f'../generated{id}.mp4', [img_as_ubyte(frame) for frame in predictions])
  videonote = open(f'../generated{id}.mp4', 'rb')
  bot.send_video_note(id, videonote)

bot.polling()

100%|██████████| 161/161 [00:07<00:00, 20.25it/s]
